<a href="https://colab.research.google.com/github/abdulwasaeee/reinforcement-learning-course/blob/main/PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 4.1 MB/s eta 0:00:00


In [20]:
import os
import time

import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecMonitor

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [22]:
id = "CartPole-v1"
n = 4
timesteps = 100000
learningrate = 2.5e-4
gamma = 0.99

# PPO parameters
n_steps = 128             # Buffer size (steps taken to fill buffer)
n_epochs = 4              # number of backpropogation steps

In [28]:
# 1. Create vectorized environment (handles parallel running automatically)
# We wrap it with VecMonitor to track returns easily
vec_env = make_vec_env(id, n_envs=n, seed=None)
vec_env = VecMonitor(vec_env, filename=None)

# 2. Initialize the PPO model
# We pass the collected hyperparameters directly to the PPO class
model = PPO(
    "MlpPolicy",
    vec_env,
    learning_rate=learningrate,
    gamma=gamma,
    n_steps=n_steps,
    n_epochs=n_epochs,
    batch_size=n*n_steps, # Total batch size is steps * n_envs
    verbose=1,
    seed=None
)

print("Model Initialized.")

Using cpu device
Model Initialized.


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/vec_env/vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(


In [29]:
print("Starting training...")
start_time = time.time()

# SB3 handles the data collection, GAE, and policy updates internally
model.learn(total_timesteps=timesteps)

end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")

Starting training...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.9     |
|    ep_rew_mean     | 21.9     |
| time/              |          |
|    fps             | 4082     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 512      |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 22.8          |
|    ep_rew_mean          | 22.8          |
| time/                   |               |
|    fps                  | 3713          |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 4.1653402e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.693        |
|    explained_variance

In [30]:
# Save the model
model_path = "ppo_cartpole_sb3.zip"
model.save(model_path)
print(f"Model saved to {model_path}")

# Load the model for evaluation
model = PPO.load(model_path)

# --- Simple Evaluation ---
envs_eval = gym.make(id, render_mode="rgb_array")
eval_returns = []

for i in range(10):
    obs, info = envs_eval.reset()
    done = False
    truncated = False
    episode_return = 0

    while not done and not truncated:
        # Get action from the trained model
        action, _ = model.predict(obs, deterministic=True)

        # Step environment
        obs, reward, done, truncated, info = envs_eval.step(action)
        episode_return += reward

    eval_returns.append(episode_return)
    print(f"Evaluation Episode {i+1}: Return = {episode_return}")

envs_eval.close()
print(f"\nAverage evaluation return over 10 episodes: {np.mean(eval_returns):.2f}")

Model saved to ppo_cartpole_sb3.zip
Evaluation Episode 1: Return = 500.0
Evaluation Episode 2: Return = 500.0
Evaluation Episode 3: Return = 341.0
Evaluation Episode 4: Return = 391.0
Evaluation Episode 5: Return = 427.0
Evaluation Episode 6: Return = 500.0
Evaluation Episode 7: Return = 500.0
Evaluation Episode 8: Return = 222.0
Evaluation Episode 9: Return = 343.0
Evaluation Episode 10: Return = 499.0

Average evaluation return over 10 episodes: 422.30
